# Задание

У каждого есть свой набор данный, который можно найти в ведомости на вкладке "Логистическая регрессия":

https://clck.ru/32BVWe

Скачиваете свой датасет с kaggle (для этого нужно авторизоваться с гугл аккаунт и скачать датасет).

**Ваша задача:**
1. Исследовать данные
2. Создать тренировочную и тестовую подвыборки
3. Построить модель логистической регрессии по своему набору данных (т.е. создать модель и обучить ее на тренировочной выборке)
4. Получить коэффиценты уравнения логистической регрессии (т.е. уравнение разделяющей поверхности). Сделать вывод какие из признаков оказались наиболее значимыми, какие нет и почему
5. Написать уравнение логистической регрессии.
6. Вывести confusion matrix для получившейся модели.
7. Вручную посчитать метрики: accuracy, recall, precision. f1.
8. Посчитать метрики с помощью библиотеки sklearn и сравнить с оценками из п.7.
9. Сделать письменный вывод получившихся результатов (хорошие или плохие получлись метрики и почему).

ВСЕ графики должны быть подписанны.
Ваша задача получить метрики как можно лучше.

In [65]:
%matplotlib inline
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from collections import Counter 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# NEW 
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import metrics

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from google.colab import files
import seaborn as sn

In [20]:
df = pd.read_csv("income_evaluation.csv")

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [22]:
df.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income'],
      dtype='object')

In [27]:
df[" occupation"].value_counts()

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name:  occupation, dtype: int64

In [37]:
def translate(df, col):
  translator = {}
  for i, item in enumerate(set(df[col])):
    translator[item] = i 
  df[col] = df[col].apply(lambda item: translator[item])
  return df

In [33]:
def prepare(df):
  df = translate(df, " sex")
  df = translate(df, " native-country")
  df = translate(df, " education")
  df = translate(df, " marital-status")
  df = translate(df, " income")
  df = translate(df, " workclass")
  df = translate(df, " occupation")
  df = translate(df, " relationship")
  df = translate(df, " race")
  return df

In [38]:
df = prepare(df)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,5,77516,12,13,1,8,3,1,1,2174,0,40,19,0
1,50,8,83311,12,13,6,14,4,1,1,0,0,13,19,0
2,38,3,215646,2,9,0,11,3,1,1,0,0,40,19,0
3,53,3,234721,10,7,6,11,4,4,1,0,0,40,19,0
4,28,3,338409,12,13,6,9,1,4,0,0,0,40,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,3,257302,4,12,6,10,1,1,0,0,0,38,19,0
32557,40,3,154374,2,9,6,0,4,1,1,0,0,40,19,1
32558,58,3,151910,2,9,2,8,5,1,0,0,0,40,19,0
32559,22,3,201490,2,9,1,8,0,1,1,0,0,20,19,0


In [48]:
X = df.drop([' income'], axis=1)
y = df[' income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
5514,33,4,198183,12,13,1,9,3,1,0,0,0,50,19
19777,36,3,86459,14,11,6,14,4,1,1,0,1887,50,19
10781,58,8,203039,3,5,3,5,3,1,1,0,0,40,19
32240,21,3,180190,14,11,6,6,4,1,1,0,0,46,19
9876,27,3,279872,7,10,0,12,3,1,1,0,0,40,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,47,3,359461,12,13,6,5,4,1,1,0,0,40,19
5390,31,3,147215,11,8,0,12,5,1,0,0,0,21,19
860,18,3,216284,10,7,1,8,0,1,0,0,0,20,19
15795,50,8,54261,2,9,6,6,4,1,1,0,0,84,19


In [88]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [89]:
print('y =', round(lr.intercept_[0], 4), end = ' ')
for i in range(len(lr.coef_[0])):
  print('+', round(lr.coef_[0][i], 4), f'* {X_train.columns.values[i]}',  end = ' ')

y = -0.0083 + 0.0249 * age + -0.0089 *  workclass + -0.0 *  fnlwgt + -0.0034 *  education + 0.0048 *  education-num + 0.0977 *  marital-status + -0.0129 *  occupation + 0.0123 *  relationship + -0.0187 *  race + 0.0064 *  sex + 0.0004 *  capital-gain + 0.0008 *  capital-loss + 0.0104 *  hours-per-week + -0.1535 *  native-country 

Возраст является важным фактором так как пожилие люди менее способны выполнять работу, в то время пол не является важным критерием тк любую работу могут сделать все одинаково не зависимо от пола 


In [90]:
y_pred = lr.predict(X_test)

In [91]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)

TN: 4657
FP: 285
FN: 1065
TP: 506


In [92]:
# Accuracy
(tp + tn) / (tp + tn + fp + fn)

0.7927222478120681

In [93]:
accuracy_score(y_test, y_pred)

0.7927222478120681

In [94]:
# Recall 
recall = tp / (tp + fn)
recall

0.3220878421387651

In [95]:
recall_score(y_test, y_pred)

0.3220878421387651

In [96]:
# Precision 
precision = tp / (tp + fp)
precision

0.6396965865992414

In [97]:
precision_score(y_test, y_pred)

0.6396965865992414

In [98]:
# F1
f1 = 2 * precision * recall / (recall + precision)
f1

0.4284504657070279

In [99]:
f1_score(y_test, y_pred)

0.4284504657070279

Метрики Accuracy и Precision достаточно высокие, в то время как метрики Recall и F1 низкие. Это говорит о том, что данные несбалансированные 